# StyleTTS2 Training on Google Colab
## Train Cigdem TTS Model

This notebook will help you train your Turkish TTS model on Google Colab with free GPU.

## Step 1: Check GPU and Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi
import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Step 2: Clone StyleTTS2 Repository

In [ ]:
# Clone your repository
!git clone https://github.com/ElkhanAbbasov/TTS_Cigdem.git
%cd TTS_Cigdem

# Verify we're in the right place
!pwd
!ls -la

## Step 3: Install Dependencies

In [ ]:
# Install required packages (INCLUDING einops-exts and click)
!pip install -q SoundFile torchaudio munch pydub pyyaml librosa nltk matplotlib accelerate transformers einops einops-exts tqdm click
!pip install -q git+https://github.com/resemble-ai/monotonic_align.git

print("✅ All dependencies installed!")

## Step 4: Mount Google Drive (for saving checkpoints)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create checkpoint backup folder in Drive
!mkdir -p '/content/drive/MyDrive/Cigdem_TTS_Checkpoints'
print("✅ Google Drive mounted!")

## Step 5: Verify Training Data

In [ ]:
# Download pretrained models from StyleTTS2
import os
import gdown

print("📥 Downloading pretrained StyleTTS2 models...")
print("This may take 5-10 minutes depending on connection speed.\n")

# Create Models directory if it doesn't exist
os.makedirs("Models", exist_ok=True)

# Download the pretrained model (LibriTTS base model)
# This is the base model needed for fine-tuning
print("Downloading base pretrained model...")
model_url = "https://huggingface.co/yl4579/StyleTTS2-LibriTTS/resolve/main/Models/LibriTTS/epochs_2nd_00020.pth"
model_path = "Models/LibriTTS_pretrained.pth"

# Use wget instead of gdown for HuggingFace
!wget -q --show-progress -O {model_path} {model_url}

if os.path.exists(model_path) and os.path.getsize(model_path) > 100000:
    print(f"✅ Base model downloaded: {model_path}")
    print(f"   Size: {os.path.getsize(model_path) / 1024 / 1024:.1f} MB")
else:
    print("❌ Download failed! Trying alternative method...")
    # Try with curl as backup
    !curl -L -o {model_path} {model_url}
    
if not os.path.exists(model_path):
    print("\n⚠️ Manual download needed:")
    print(f"   1. Download from: {model_url}")
    print(f"   2. Upload to: {model_path}")
else:
    print("\n✅ All pretrained models ready for fine-tuning!")

## Step 5.5: Download Required Pretrained Models

StyleTTS2 requires base pretrained models for fine-tuning.

In [ ]:
# Check if training data exists
import os

train_list = "Data/my_train_list.txt"
if os.path.exists(train_list):
    with open(train_list, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    print(f"✅ Training data found: {len(lines)} samples")
    print("\nFirst 3 samples:")
    for line in lines[:3]:
        print(f"  {line.strip()}")
    
    # Quick check that all audio files can be loaded
    print("\n🔍 Checking all audio files...")
    import soundfile as sf
    all_valid = True
    for i, line in enumerate(lines, 1):
        path = line.strip().split('|')[0]
        try:
            wave, sr = sf.read(path)
            print(f"  ✓ {i}: {path} ({sr} Hz)")
        except Exception as e:
            print(f"  ✗ {i}: {path} - ERROR: {e}")
            all_valid = False
    
    if all_valid:
        print("\n✅ All audio files can be loaded!")
    else:
        print("\n⚠️ Some audio files have issues!")
else:
    print("❌ Training data not found!")
    print("\n📂 Current directory:", os.getcwd())
    print("\n📋 Directory contents:")
    !ls -la
    print("\n📋 Looking for Data folder:")
    !ls -la Data/ 2>/dev/null || echo "Data folder not found"
    print("\n⚠️ Make sure the repository was cloned correctly!")

## Step 6: Configure Training Settings

Update the config to use the pretrained model and prepare for training.

## Step 7: Start Training

**IMPORTANT:** 
- Free Colab sessions last ~12 hours max
- Training will run continuously
- Checkpoints saved every 2 epochs to Models/Cigdem_TTS/
- Backup to Google Drive regularly!
- If disconnected, re-run from Step 6 to resume from last checkpoint

In [ ]:
# Configure training settings
import yaml
import os

config_path = "Configs/config_ft.yml"
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Set the pretrained model path
pretrained_model_path = "Models/LibriTTS_pretrained.pth"

if os.path.exists(pretrained_model_path):
    print(f"✅ Found pretrained model: {pretrained_model_path}")
    config['pretrained_model'] = pretrained_model_path
    config['second_stage_load_pretrained'] = True
    config['load_only_params'] = True  # Load only model weights, not optimizer state
    print("   Will load pretrained weights for fine-tuning")
else:
    print(f"⚠️ Pretrained model not found: {pretrained_model_path}")
    print("   Please run Step 5.5 to download the model")
    config['second_stage_load_pretrained'] = False
    config['pretrained_model'] = ""

# Ensure proper training settings
config['batch_size'] = 2  # Small batch size for fine-tuning
config['epochs'] = 100
config['save_freq'] = 2

# Save updated config
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("\n📋 Training Configuration:")
print(f"  Pretrained model: {config.get('pretrained_model', 'None')}")
print(f"  Load pretrained: {config.get('second_stage_load_pretrained', False)}")
print(f"  Batch size: {config.get('batch_size', 'N/A')}")
print(f"  Epochs: {config.get('epochs', 'N/A')}")
print(f"  Save frequency: {config.get('save_freq', 'N/A')} epochs")
print(f"  Log dir: {config.get('log_dir', 'N/A')}")

if config.get('second_stage_load_pretrained'):
    print("\n✅ Ready to start training!")
else:
    print("\n❌ Cannot start training without pretrained model!")

In [ ]:
# Start training - runs continuously until complete or stopped
# If this is a resume, it will automatically load the latest checkpoint
!python train_finetune.py --config_path Configs/config_ft.yml

## Step 8: Backup Checkpoints to Google Drive (Run periodically)

**Run this cell every 2-3 hours while training runs above!**

In [ ]:
# Copy checkpoints to Google Drive
!cp Models/Cigdem_TTS/epoch_*.pth '/content/drive/MyDrive/Cigdem_TTS_Checkpoints/' 2>/dev/null || echo "No checkpoints yet"
!cp Models/Cigdem_TTS/train.log '/content/drive/MyDrive/Cigdem_TTS_Checkpoints/' 2>/dev/null || echo "No log yet"

# List backed up files
!ls -lh '/content/drive/MyDrive/Cigdem_TTS_Checkpoints/'
print("\n✅ Checkpoints backed up!")

## Step 9: View Training Progress

In [ ]:
# Display last 20 lines of training log
!tail -n 20 Models/Cigdem_TTS/train.log

## Step 9: Download Results (After epoch 20+)

Download checkpoints to test locally

In [ ]:
# Zip checkpoints for download
!zip -r cigdem_tts_checkpoints.zip Models/Cigdem_TTS/epoch_*.pth Models/Cigdem_TTS/train.log

from google.colab import files
files.download('cigdem_tts_checkpoints.zip')

print("✅ Ready to download!")

---

## 📝 Important Notes:

### Training Process:
- Training runs **continuously** once started (Step 6)
- Goes from epoch 1 → 100 automatically
- Each epoch takes ~2-3 minutes
- **Don't re-run Step 6** unless training stops!

### When to Test:
- **Epoch 20+**: First test (may still be noisy)
- **Epoch 30+**: Better quality expected
- **Epoch 50+**: Good quality for your voice

### If Colab Disconnects:
1. Restore checkpoints from Google Drive:
   ```python
   !cp '/content/drive/MyDrive/Cigdem_TTS_Checkpoints/epoch_*.pth' Models/Cigdem_TTS/
   ```
2. Re-run Step 6 (training cell)
3. It will automatically resume from last checkpoint!